In [110]:
import os
from os import listdir
import random

<font size=6>__Data__</font>

In [1]:
import pandas as pd
import json

In [2]:
df = pd.read_csv("D:/PythonFile/DataAnalysis/data/AustraliaFire/fire_archive_M6_96619.csv")

test = df.head()
df.head()

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type
0,-11.8070,142.0583,313.0,1.0,1.0,2019-08-01,56,Terra,MODIS,48,6.3,297.3,6.6,D,0
1,-11.7924,142.0850,319.3,1.0,1.0,2019-08-01,56,Terra,MODIS,71,6.3,297.3,11.3,D,0
2,-12.8398,132.8744,311.6,3.1,1.7,2019-08-01,57,Terra,MODIS,42,6.3,298.7,23.1,D,0
3,-14.4306,143.3035,310.1,1.1,1.1,2019-08-01,57,Terra,MODIS,33,6.3,296.1,6.5,D,0
4,-12.4953,131.4897,310.3,4.0,1.9,2019-08-01,57,Terra,MODIS,36,6.3,298.8,27.6,D,0


###  构造位置json文件

In [3]:
# 统计表格天数
days = set()
for d in df['acq_date']:
    days.add(d)

print(len(days))
print(days)

61
{'2019-08-22', '2019-08-03', '2019-09-27', '2019-09-02', '2019-08-28', '2019-09-18', '2019-08-09', '2019-09-21', '2019-08-10', '2019-09-04', '2019-08-30', '2019-08-12', '2019-08-21', '2019-08-27', '2019-09-29', '2019-09-08', '2019-08-29', '2019-08-14', '2019-09-16', '2019-09-20', '2019-09-17', '2019-08-31', '2019-08-08', '2019-08-02', '2019-08-23', '2019-09-11', '2019-09-06', '2019-09-22', '2019-09-14', '2019-09-28', '2019-09-07', '2019-09-15', '2019-08-07', '2019-09-23', '2019-09-03', '2019-08-04', '2019-08-15', '2019-09-13', '2019-08-16', '2019-09-30', '2019-08-13', '2019-09-10', '2019-09-25', '2019-08-26', '2019-09-05', '2019-08-18', '2019-08-01', '2019-09-19', '2019-09-26', '2019-08-20', '2019-09-01', '2019-08-11', '2019-09-12', '2019-08-24', '2019-08-25', '2019-09-09', '2019-08-05', '2019-08-17', '2019-08-06', '2019-09-24', '2019-08-19'}


In [17]:
# 按天、白天晚上读取起火点位置的经纬度
for d in days:
    day = df[df['acq_date'] == d]
    
    day_d = day[day['daynight'] == "D"]
    day_n = day[day['daynight'] == "N"]
    
#   读取白天的位置经纬度
    try:
        data_d = {}
        for i in range(day_d.index[0],day_d.index[-1]+1):
            try:
                data_d["Fire_{}".format(i)] = [day_d['longitude'][i],day_d['latitude'][i]]
            except:
                continue
        
        with open("day_{}_d.json".format(d.replace('-',"_")),'w') as f:
            json.dump(data_d,f)
            print("{} success".format(d.replace("-",'_')))
    except:
        print('{} D数据 failure'.format(d))
    
#   读取黑夜的位置经纬度
    try:
        data_n = {}
        for i in range(day_n.index[0],day_n.index[-1]+1):
            data_n["Fire_{}".format(i)] = [day_n['longitude'][i],day_n['latitude'][i]]
        
        with open("day_{}_n.json".format(d.replace('-',"_")),'w')as f:
            json.dump(data_n,f)
            print("{} success".format(d.replace("-",'_')))
    except:
        print('{} N数据 failure'.format(d))

2019_08_22 success
2019_08_22 success
2019_08_03 success
2019_08_03 success
2019_09_27 success
2019_09_27 success
2019_09_02 success
2019_09_02 success
2019_08_28 success
2019_08_28 success
2019_09_18 success
2019_09_18 success
2019_08_09 success
2019_08_09 success
2019_09_21 success
2019_09_21 success
2019_08_10 success
2019_08_10 success
2019_09_04 success
2019_09_04 success
2019_08_30 success
2019_08_30 success
2019_08_12 success
2019_08_12 success
2019_08_21 success
2019_08_21 success
2019_08_27 success
2019_08_27 success
2019_09_29 success
2019_09_29 success
2019_09_08 success
2019_09_08 success
2019_08_29 success
2019_08_29 success
2019_08_14 success
2019_08_14 success
2019_09_16 success
2019_09_16 success
2019_09_20 success
2019_09_20 success
2019_09_17 success
2019_09_17 success
2019_08_31 success
2019_08_31 success
2019_08_08 success
2019_08_08 success
2019_08_02 success
2019_08_02 success
2019_08_23 success
2019_08_23 success
2019_09_11 success
2019_09_11 success
2019_09_06 s

<font size=6>__Pyecharts__</font>

In [29]:
from pyecharts import options as opts
from pyecharts.charts import Geo
from pyecharts.faker import Faker 
from pyecharts.globals import ChartType

from pyecharts.datasets import COORDINATES
COORDINATES.cutoff = 0.75

path = "d:/pythonfile/dataanalysis/australia_fire/"
filelist = listdir(path)

for f in filelist:
    loc_d = []
    loc_n = []
    if f[-6:] == 'd.json':
        loc = open(path+f)
        data = json.load(loc)
        for k in data:
            loc_d.append(k)
        c = (geo()
            .add_schema("澳大利亚")
            .add_coordinate(f)
            .add("day",[list(z) for z in zip(loc_d,count)],'scatter')
             
            .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
            .ser_globel_opts(title_opts.opts.TitleOpts(title="Fire plot"))
             
             .render("{}.html".format(f[0:]))
            )
        
    if f[-6:] == 'n.json':
        loc = open(path+f)
        data = json.load(loc)

        

In [114]:
loc_d = open("d:/pythonfile/dataanalysis/australia_fire/day_2019_08_01_d.json")
data = json.load(loc_d)

loc_n = open('d:/pythonfile/dataanalysis/australia_fire/day_2019_08_01_n.json')
data_n = json.load(loc_n)

ls_d = []
for k in data:
    ls_d.append(k)
    
ls_n = []
for k in data_n:
    ls_n.append(k)

count = []
for i in range(0,420):
    w = random.randint(10,1000)
    count.append(w)

<ipython-input-114-ae3e2653c3e9>:1: ResourceWarning:

unclosed file <_io.TextIOWrapper name='d:/pythonfile/dataanalysis/australia_fire/day_2019_08_01_d.json' mode='r' encoding='cp936'>

<ipython-input-114-ae3e2653c3e9>:4: ResourceWarning:

unclosed file <_io.TextIOWrapper name='d:/pythonfile/dataanalysis/australia_fire/day_2019_08_01_n.json' mode='r' encoding='cp936'>



In [103]:
c =( Geo()
    .add_schema(maptype='澳大利亚')
    .add_coordinate_json("day_2019_08_01_d.json")
    .add_coordinate_json("day_2019_08_01_n.json")
    .add("day",[list(z) for z in zip(ls_d,count)],"effectScatter")
    .add("night",[list(z) for z in zip(ls_n,count)],'scatter')
    
    .set_series_opts(label_opts=opts.LabelOpts(is_show=True))
    .set_global_opts(title_opts=opts.TitleOpts(title="Geo"))
    .render("2019_0801.html")
   )

# c.render_notebook()
print("successfully")

successfully


d:\anaconda\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.

d:\anaconda\lib\asyncio\events.py:81: DeprecationWarning:

`run_cell_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.

d:\anaconda\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning:

pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)

